<a href="https://colab.research.google.com/github/AkashKoley012/Computer-Vision-Projects/blob/main/Face%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d andrewmvd/face-mask-detection

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection
License(s): CC0-1.0
 97% 384M/398M [00:05<00:00, 16.7MB/s]
100% 398M/398M [00:05<00:00, 74.5MB/s]


In [2]:
import zipfile
with zipfile.ZipFile('/content/face-mask-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('face-mask')

In [48]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from PIL import Image

import cv2
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [4]:
os.remove('/content/face-mask-detection.zip')

In [11]:
img_path = '/content/face-mask/images/'
loc_path = '/content/face-mask/annotations/'

In [42]:
images = []
labels = []

dic = {'without_mask': 0, 'with_mask': 1, 'mask_weared_incorrect': 2}

for file in tqdm(os.listdir(img_path)):
  img = cv2.imread(img_path + file)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  images.append(img)

  with open(loc_path + file.split('.')[0] + '.xml', 'r') as f:
    xml = f.read()
  xml = BeautifulSoup(xml, 'xml')
  label = []
  for obj in xml.find_all('object'):
    name = obj.find('name').text
    bndbox = obj.find('bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)
    label.append([dic[name], xmin, ymin, xmax, ymax])
  labels.append(label)

100%|██████████| 853/853 [00:17<00:00, 47.81it/s]


In [43]:
images[0], labels[0]

(array([[[184, 180, 172],
         [183, 180, 172],
         [183, 179, 171],
         ...,
         [189, 168, 152],
         [179, 157, 137],
         [182, 161, 138]],
 
        [[184, 180, 172],
         [183, 180, 171],
         [183, 179, 170],
         ...,
         [186, 163, 146],
         [178, 154, 132],
         [181, 158, 134]],
 
        [[184, 180, 170],
         [184, 180, 170],
         [183, 179, 169],
         ...,
         [180, 151, 128],
         [177, 149, 122],
         [180, 152, 124]],
 
        ...,
 
        [[ 32,  32,  34],
         [ 33,  33,  35],
         [ 35,  35,  37],
         ...,
         [ 90,  89,  94],
         [ 93,  91,  97],
         [ 99,  96, 103]],
 
        [[ 34,  34,  36],
         [ 30,  30,  32],
         [ 34,  34,  36],
         ...,
         [ 92,  92,  97],
         [ 89,  87,  93],
         [ 92,  90,  96]],
 
        [[ 29,  29,  31],
         [ 38,  38,  39],
         [ 35,  35,  37],
         ...,
         [ 96,  95, 101],
  

In [44]:
df = pd.DataFrame({'image': images, 'label': labels})

In [47]:
df.iloc[0]

,0
image,"[[[184, 180, 172], [183, 180, 172], [183, 179,..."
label,"[[0, 101, 73, 108, 82], [1, 123, 67, 143, 83],..."


In [41]:
# dic = {'other':0}

# for i in range(len(df['label'])):
#   for j in range(len(df['label'][i])):
#     if df['label'][i][j][0] not in dic:
#       dic[df['label'][i][j][0]] = 1
#     else:
#       dic[df['label'][i][j][0]] += 1

#print(dic) #{'other': 0, 'without_mask': 717, 'with_mask': 3232, 'mask_weared_incorrect': 123}

{'other': 0, 'without_mask': 717, 'with_mask': 3232, 'mask_weared_incorrect': 123}


In [ ]:
split = int(0.8*(len(df)))
train_df = df.iloc[:int(split)]
test_df = df.iloc[int(split):]

# Dataset & DataLoader

In [ ]:
class FaceMaskDataset(torch.utils.data.Dataset):

  def __init__(self, df, transforms=None):
    self.df = df
    self.transforms = transforms

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    image = self.df.iloc[index]['image']
    label = self.df.iloc[index]['label']

    bbox = label[1:].values.astype("float32")

    if self.transform:
      image = self.transform(image)

    return image, torch.tensor(bbox)

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = FaceMaskDataset(train_df, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = FaceMaskDataset(test_df, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)